<a href="https://colab.research.google.com/github/Dragon116rus/nlp_project/blob/master/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim --upgrade

     |████████████████████████████████| 24.2MB 370kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [0]:
import pandas as pd
import numpy as np
import nltk
from tqdm import tqdm_notebook as tqdm
# import tensorflow as tf
from gensim.models import FastText
from gensim.test.utils import datapath
import gc
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

# Loading dataset



In [3]:
!git clone https://github.com/leondz/emerging_entities_17.git

Cloning into 'emerging_entities_17'...
remote: Enumerating objects: 53, done.
remote: Total 53 (delta 0), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (53/53), done.


In [0]:
df_train = pd.read_csv("emerging_entities_17/wnut17train.conll", sep="\t", header=None)
df_valid = pd.read_csv("emerging_entities_17/emerging.dev.conll", sep="\t", header=None)
# df_test = pd.read_csv("emerging_entities_17/emerging.test.conll", sep="\t", header=None)

In [0]:
with open("emerging_entities_17/emerging.test") as f:
  lines = f.readlines()
lines = [i.rstrip() for i in lines][:-1]
df_test = pd.DataFrame(lines)

# Loading fasttext embedding

In [6]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip

--2019-11-07 16:44:25--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip’

crawl-300d-2M-subwo 100%[===================>]   5.43G  12.7MB/s    in 7m 27s  

2019-11-07 16:51:53 (12.4 MB/s) - ‘crawl-300d-2M-subword.zip’ saved [5828358084/5828358084]



In [7]:
!unzip crawl-300d-2M-subword.zip

Archive:  crawl-300d-2M-subword.zip
  inflating: crawl-300d-2M-subword.vec  
  inflating: crawl-300d-2M-subword.bin  


In [0]:
import gensim

In [0]:
fasttext_emb = gensim.models.fasttext.load_facebook_vectors("crawl-300d-2M-subword.bin", encoding='utf-8')

#Preprocessing

In [0]:
df_train.dropna(inplace=True)

In [0]:
df_train["vector"] = df_train[0].apply(lambda x: fasttext_emb[str(x)])
df_test["vector"] = df_test[0].apply(lambda x: fasttext_emb[str(x)])
df_valid["vector"] = df_valid[0].apply(lambda x: fasttext_emb[str(x)])


In [0]:
unique = df_train[1].unique()
unique_map = {k:v for v, k in enumerate(unique)}

In [0]:
NB_CLASSES = len(unique)

In [0]:
df_train['labels'] = df_train[1].map(unique_map)
df_valid['labels'] = df_valid[1].map(unique_map)

#Model definition

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [0]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        self.linear0 = nn.Linear(in_features=300, out_features=500 )
#         self.linear1 = nn.Linear(in_features=4000, out_features=4000 )
#         self.linear2 = nn.Linear(in_features=1500, out_features=1500 )
        self.linear = nn.Linear(in_features=500, out_features=NB_CLASSES )
        self.dropout0 = nn.Dropout(0.5)
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        self.dropout3 = nn.Dropout(0.5)

    def forward(self, x, debug=False):
        x = self.linear0(x)
        x = self.dropout0(x)
        x = F.relu(x)
#         x = self.linear1(x)
#         x = self.dropout1(x)
#         x = F.leaky_relu(x)
#         x = self.linear2(x)
#         x = self.dropout2(x)
#         x = F.leaky_relu(x)
        
        x = self.linear(x)
#         x = self.dropout3(x)
        x = torch.softmax(x, 1)
        return x


#Training

In [0]:
X_train, y_train = df_train['vector'].values, df_train['labels'].values

In [0]:
X_val, y_val = df_valid['vector'].values, df_valid['labels'].values

In [0]:
X_train = np.stack(X_train)
X_val = np.stack(X_val)

In [0]:
def score(y_true, y_pred, threshold=0.3):
    # pred = y_pred.argmax(axis=1)
    conf_matrix = confusion_matrix(y_true, y_pred)
    print(conf_matrix)
    print("accuracy: ", accuracy_score(y_true, y_pred))
    print("weighted f1: ", f1_score(y_true, y_pred, average="weighted"))
    y_true_non_zero = y_true[y_true!=0]
    y_pred_non_zero =y_pred[y_true!=0]
    print("weighted f1 non zero: ", f1_score(y_true_non_zero, y_pred_non_zero, average="weighted"))
    return None

In [0]:
def validate(net, X_val=X_val, y_val=y_val,batch_size=512, threshold=0.1):
  net.eval()
  predict_all = np.zeros_like((y_val))
  true_labels_all =  np.zeros_like((y_val))
  for i in range(1, len(X_val), batch_size):
        # get the inputs; data is a list of [inputs, labels]
        from_, to_ = i - 1, min(len(y_val), i - 1 + batch_size)
        inputs, labels = X_val[from_: to_], y_val[from_: to_]
        inputs = torch.tensor(inputs).cuda()
        batch_size_tmp = to_ - from_
        predict = net(inputs).cpu().detach().numpy()
        predict_all[from_: to_] = predict.argmax(axis=1)
        true_labels_all[from_: to_] = labels
  # threshold = 0.2#np.quantile(predict, 0.001)
  # print("threshold:", threshold)
  
  net.train()
  return score(true_labels_all, predict_all)

In [43]:
validate(net)

[[4024    0    0    0    0  182    0   76    0  366    0    0   87]
 [   6    1    0    0    0   16    0    5    0    1    0    0    1]
 [   4    0    0    0    0    2    0    2    0    7    0    0    0]
 [   2    0    0    0    0    5    0    1    0    0    0    0    0]
 [   3    0    0    0    0    6    0    1    0    0    0    0    0]
 [   1    0    0    0    0  168    0    0    0    1    0    0    0]
 [   4    0    0    0    0   16    0    8    0    6    0    0    0]
 [   2    0    0    0    0    3    0   37    0    0    0    0    0]
 [   6    0    0    0    0   33    0    1    0    3    0    0    0]
 [  10    0    0    0    0    3    0    1    0   39    0    0    0]
 [   5    0    0    0    0    0    0    0    0    1    0    0    0]
 [   2    0    0    0    0    2    0    0    0    0    0    0    0]
 [   0    0    0    0    0    2    0    2    0    4    0    0   25]]
accuracy:  0.8284777156087209
weighted f1:  0.8608611186552939
weighted f1 non zero:  0.5156560685158328


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
unique_inv_count = 1/np.unique(y_train, return_counts=True)[1]
unique_inv_freq = (unique_inv_count)/unique_inv_count.sum()

In [0]:
unique_inv_count = 1/np.unique(y_train, return_counts=True)[1]
unique_inv_freq = (unique_inv_count)/len(unique_inv_count)
probas_sample = np.apply_along_axis(lambda x: unique_inv_freq[x], 0, y_train)
dataset_len = len(y_train)

In [70]:
probas_sample.sum(), len(unique_inv_count)

(12.999999999999998, 13)

In [0]:
def generator(X, y, batch_size, steps=1):

  unique_inv_count = 1/np.unique(y, return_counts=True)[1]
  unique_inv_freq = (unique_inv_count)/len(unique_inv_count)
  probas_sample = np.apply_along_axis(lambda x: unique_inv_freq[x], 0, y)
  dataset_len = len(y)

  zero_class = (y==0).nonzero()[0]
  non_zero_class = (y!=0).nonzero()[0]
  fraction = 1/8
  batch1 = int(batch_size*fraction)
  batch2 = batch_size - batch1
  for i in range(steps):
    idx = np.random.choice(dataset_len, batch1, p=probas_sample)
    yield X[idx], y[idx]

In [73]:
net = Net()
net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3000000, gamma=0.95)
net.train()


[0,    32] loss: 2.565
[[ 401  152   32  356    0   11 1283 2124    0  315    0   61    0]
 [   0    0    0    0    0    0    3   23    0    4    0    0    0]
 [   1    0    0    0    0    0    2    8    0    4    0    0    0]
 [   0    0    0    0    0    0    2    2    0    4    0    0    0]
 [   0    0    0    0    0    0    1    9    0    0    0    0    0]
 [   0    0    0    0    0    0   18  134    0   18    0    0    0]
 [   0    0    0    0    0    0    7   24    0    3    0    0    0]
 [   0    0    0    1    0    0    6   30    0    5    0    0    0]
 [   0    0    0    0    0    0    3   31    0    9    0    0    0]
 [   1    3    0    0    0    0    6   32    0    8    0    3    0]
 [   0    0    0    0    0    0    2    3    0    1    0    0    0]
 [   0    0    0    0    0    0    0    2    0    0    0    2    0]
 [   7    2    0    0    0    0    4   16    0    3    0    1    0]]
accuracy:  0.08643642677985723
weighted f1:  0.1430955100607591
weighted f1 non zero:  0.048

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[[ 542  115   10    1    0    0 2765 1161    0   81    0   60    0]
 [   0    1    0    0    0    0   10   17    0    2    0    0    0]
 [   1    0    0    0    0    0    5    6    0    3    0    0    0]
 [   0    0    0    0    0    0    3    4    0    1    0    0    0]
 [   0    0    0    0    0    0    2    8    0    0    0    0    0]
 [   0    0    0    0    0    0   59  106    0    5    0    0    0]
 [   0    0    0    0    0    0   14   19    0    1    0    0    0]
 [   0    0    0    1    0    0   16   24    0    1    0    0    0]
 [   0    0    0    0    0    0   16   25    0    2    0    0    0]
 [   1    2    0    0    0    0   15   25    0    7    0    3    0]
 [   1    0    0    0    0    0    2    2    0    1    0    0    0]
 [   0    0    0    0    0    0    1    1    0    0    0    2    0]
 [   7    1    0    0    0    0    8   15    0    0    0    2    0]]
accuracy:  0.11383368705382983
weighted f1:  0.18867165792781618
weighted f1 non zero:  0.05577120097036616
score N

In [74]:

batch_size = 128
gen_data = generator(X_val, y_val, batch_size)
for epoch in range(400):  # loop over the dataset multiple times
    
    running_loss = 0.0
    iter_count = 1
    gen_data = generator(X_val, y_val, batch_size)
    for i, (inputs, labels) in enumerate(gen_data, 32):
        
        inputs, labels = torch.tensor(inputs).cuda(), torch.tensor(labels).cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        # print statistics
        running_loss += loss.item()
        # if i % iter_count <= 8:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch, i, running_loss / iter_count * batch_size))
        #     running_loss = 0.0
    
    print('[%d, %5d] loss: %.3f' %
            (epoch, i, running_loss ))
    gc.collect()
    torch.cuda.empty_cache()  
    torch.save(net.state_dict(), "model_epoch%s.dms"%epoch)
    f1 = validate(net)
    print("score" , f1)
    print(scheduler.get_lr())
    # validate(net, X_train, y_train)
print('Finished Training')

[0,    32] loss: 2.099
[[3467  136   51  180   64   22    0  140  139    0  146    4  386]
 [   0   25    1    1    2    1    0    0    0    0    0    0    0]
 [   1    2   12    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    8    0    0    0    0    0    0    0    0    0]
 [   1    0    0    0    9    0    0    0    0    0    0    0    0]
 [   1    9    1    3    7   98    0   11   35    0    1    0    4]
 [   4    6    1    2    0    8    0    8    3    0    1    0    1]
 [   0    1    0    0    0    1    0   34    1    0    1    0    4]
 [   5    4    0    1    1    8    0    2   19    0    2    0    1]
 [  15    3    5    1    2    3    0    4    3    1    6    0   10]
 [   0    0    0    0    0    0    0    0    0    0    6    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    4    0]
 [   0    1    0    0    0    0    0    2    0    0    1    0   29]]
accuracy:  0.7161875361759599
weighted f1:  0.7984238767345362
weighted f1 non zero:  0.5155

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[[3477  129   51  180   65   26    0  165  133    0  144    4  361]
 [   0   25    1    1    2    1    0    0    0    0    0    0    0]
 [   1    2   12    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    8    0    0    0    0    0    0    0    0    0]
 [   1    0    0    0    9    0    0    0    0    0    0    0    0]
 [   1    8    1    3    7  100    0   13   32    0    1    0    4]
 [   4    6    1    2    0    9    0    8    2    0    1    0    1]
 [   0    1    0    0    0    1    0   34    1    0    1    0    4]
 [   5    4    0    1    1    9    0    2   18    0    2    0    1]
 [  15    3    5    1    2    5    0    4    1    1    6    0   10]
 [   0    0    0    0    0    0    0    0    0    0    6    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    4    0]
 [   0    1    0    0    0    0    0    3    0    0    1    0   28]]
accuracy:  0.7181169207022959
weighted f1:  0.7993046743610346
weighted f1 non zero:  0.5136376638635654
score None

#Predict

In [0]:
X_valid = df_valid['vector'].values
X_valid= np.stack(X_valid)

In [0]:
inputs = torch.tensor(X_valid).cuda()

In [0]:
pred = net(inputs).detach().cpu().numpy().argmax(axis=1)

In [0]:
(y_val[y_val!=0] == pred[y_val!=0]).mean()

0.6026785714285714